# Attempt to Convert Field to Overhead Model

We'll start simple. We'll use the PyImageSearch guy's four_point_transform template to see what we can get

In [7]:
import cv2
import numpy as np

In [8]:
def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

In [9]:
def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped

For now, since we're working with a static image, we'll select the corners ourselves. We'll use the corners of the gray playing field instead of the top of the walls because the top of the walls cuts off playing area from the bottom and adds unnecessary area at the top.

In [10]:
def transform_and_show(img, corners):
	cv2.imshow("normal", img)
	img = four_point_transform(img, corners)
	cv2.imshow("warped", img)
	cv2.waitKey()
	cv2.destroyAllWindows()

	return img

img = cv2.imread("data/raw/images/screenshot.png")
field_corners = np.array([(90, 740), (350, 240), (1040, 240), (1270, 740)])

transform_and_show(img, field_corners)

array([[[ 73,  76, 100],
        [ 67,  68,  93],
        [ 62,  62,  88],
        ...,
        [ 61,  64,  68],
        [ 55,  59,  64],
        [ 50,  55,  60]],

       [[ 75,  79,  99],
        [ 74,  76,  98],
        [ 71,  72,  94],
        ...,
        [ 62,  64,  69],
        [ 57,  60,  65],
        [ 52,  57,  61]],

       [[ 76,  80,  97],
        [ 79,  81,  99],
        [ 79,  80,  98],
        ...,
        [ 64,  66,  70],
        [ 59,  62,  66],
        [ 54,  58,  62]],

       ...,

       [[149, 150, 147],
        [120, 122, 121],
        [ 62,  64,  65],
        ...,
        [  1,   8,  15],
        [  4,  12,  18],
        [  4,  12,  18]],

       [[171, 173, 165],
        [152, 154, 150],
        [119, 121, 119],
        ...,
        [  5,  12,  20],
        [  6,  14,  20],
        [  7,  15,  19]],

       [[159, 161, 153],
        [133, 134, 129],
        [ 86,  88,  85],
        ...,
        [  5,  11,  17],
        [  5,  11,  17],
        [  5,  12,  16]]

You might notice that the image is a rectangle where it should probably be a square if we want an accurate model. So we'll change the `four_point_transform` function a bit to deal with that.

In [11]:
from sklearn.metrics.pairwise import euclidean_distances

def four_point_transform(image, pts):
	points = order_points(pts)

	# the side length is the minimum distance between any of the 4 points.
	distances = euclidean_distances(points, points)	 # Compute distances between all points

	# Get the smallest non-zero distance (0 means we compared the point with itself)
	flat = distances.flatten()
	side_length = flat[flat != 0].min()
	side_length = int(side_length)

	dst = np.array([
		[0, 0],
		[side_length - 1, 0],
		[side_length - 1, side_length - 1],
		[0, side_length - 1]], dtype = "float32")

	M = cv2.getPerspectiveTransform(points, dst)
	warped = cv2.warpPerspective(image, M, (side_length, side_length))

	return warped

In [12]:
transformed = transform_and_show(img, field_corners)
print(transformed.shape)

(550, 550, 3)


As we can see we now have a square image.